## Modelo CatBoost

## Inicialización de dependencias

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

## Carga de los datos y análisis exploratorio

In [ ]:
df = pd.read_csv('train.csv')
df.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


### Tamaño del conjunto de datos

In [ ]:
print(f"Cantidad de registros: {df.shape[0]}")
print(f"Cantidad de columnas: {df.shape[1]}")

Cantidad de registros: 692500
Cantidad de columnas: 21


### Columnas y sus tipos de datos

In [ ]:
for column in df.columns:
  print("%30s"%column, df[column].dtype)

                            ID int64
             PERIODO_ACADEMICO int64
              E_PRGM_ACADEMICO object
           E_PRGM_DEPARTAMENTO object
   E_VALORMATRICULAUNIVERSIDAD object
          E_HORASSEMANATRABAJA object
             F_ESTRATOVIVIENDA object
               F_TIENEINTERNET object
              F_EDUCACIONPADRE object
               F_TIENELAVADORA object
              F_TIENEAUTOMOVIL object
            E_PRIVADO_LIBERTAD object
         E_PAGOMATRICULAPROPIO object
             F_TIENECOMPUTADOR object
             F_TIENEINTERNET.1 object
              F_EDUCACIONMADRE object
            RENDIMIENTO_GLOBAL object
                   INDICADOR_1 float64
                   INDICADOR_2 float64
                   INDICADOR_3 float64
                   INDICADOR_4 float64


### Inspección de las columnas categóricas

In [ ]:
categorical_columns = [col for col in df.columns if not col in df._get_numeric_data()]
for column in categorical_columns:
  print("%20s"%column, np.unique(df[column].dropna()))

    E_PRGM_ACADEMICO ['3°  CICLO PROFESIONAL NEGOCIOS INTERNACIONALES'
 'ACTIVIDAD FISICA Y DEPORTE' 'ACUICULTURA' 'ADMINISTRACION'
 'ADMINISTRACION  FINANCIERA' 'ADMINISTRACION & SERVICIO'
 'ADMINISTRACION AERONAUTICA' 'ADMINISTRACION AGROPECUARIA'
 'ADMINISTRACION AMBIENTAL'
 'ADMINISTRACION AMBIENTAL Y DE LOS RECURSOS NATURALES'
 'ADMINISTRACION BANCARIA Y FINANCIERA' 'ADMINISTRACION COMERCIAL'
 'ADMINISTRACION COMERCIAL Y DE MERCADEO'
 'ADMINISTRACION DE COMERCIO EXTERIOR' 'ADMINISTRACION DE EMPRESAS'
 'ADMINISTRACION DE EMPRESAS  Y  GESTION AMBIENTAL'
 'ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES'
 'ADMINISTRACION DE EMPRESAS AGROPECUARIAS'
 'ADMINISTRACION DE EMPRESAS COMERCIALES'
 'ADMINISTRACION DE EMPRESAS EN TELECOMUNICACIONES'
 'ADMINISTRACION DE EMPRESAS TURISTICA'
 'ADMINISTRACION DE EMPRESAS TURISTICAS'
 'ADMINISTRACION DE EMPRESAS TURISTICAS Y HOTELERAS'
 'ADMINISTRACION DE EMPRESAS TURÍSTICAS Y HOTELERAS'
 'ADMINISTRACION DE EMPRESAS Y FINANZAS'
 'ADMINISTRACION DE GEST

### Valores faltantes en las columnas

In [ ]:
missing_values = df.isna().sum()
missing_values[missing_values!=0]

,0
E_VALORMATRICULAUNIVERSIDAD,6287
E_HORASSEMANATRABAJA,30857
F_ESTRATOVIVIENDA,32137
F_TIENEINTERNET,26629
F_EDUCACIONPADRE,23178
F_TIENELAVADORA,39773
F_TIENEAUTOMOVIL,43623
E_PAGOMATRICULAPROPIO,6498
F_TIENECOMPUTADOR,38103
F_TIENEINTERNET.1,26629


## Preprocesamiento de los datos

Para el preprocesamiento de los datos vamos a realizar los siguientes pasos:
* Crear una función que nos va a permitir agrupar las carreras, para reducir la cardinalidad de la variable `E_PRGR_ACADEMICO`.
* Vamos a eliminar las columnas `E_PRIVADO_LIBERTAD` y `F_TIENEINTERNET.1`. La primera no nos aporta casi información, debido a que la cantidad de personas que si son muy pocas, y la última esta repetida junto con la columna `F_TIENEINTERNET`.
* Normalizamos los strings de la variable `E_PRGR_ACADEMICO` y hacemos la agrupación por carreras.
* Imputamos los valores faltantes de las columnas. En general, para las columnas binarias se hizo el llenado con el valor 'No' y para las demás columnas se siguio la misma idea, llenando con el valor negativo que tenga la columna.
* Se hizo el mapeo de las variables categoricas a sus respectivas variables numericas.
* Para mejorar las predicciones del modelo se hizo una combinación entre target y frequency encoding para la variable `E_PRGR_ACADEMICO`. Se agregaron dos columnas adicionales para guardar estos valores.
* Siguiendo lo anterior, para poder utilizar estas columnas en la fase de test, se guardaron los valores que se calcularon (frecuencias y media relativa a la variable objetivo) en un diccionario llamado encoders, para usarlo posteriormente.

In [ ]:
def get_career_cluster(text):
    text = str(text)
    if 'INGENIERI' in text: return 'INGENIERIA'
    if 'LICENCIATURA' in text or 'PEDAGOGI' in text or 'EDUCACION' in text: return 'EDUCACION'
    if 'ADMINISTRACI' in text or 'GERENCIA' in text or 'GESTION' in text: return 'ADMINISTRACION'
    if 'CONTADURI' in text or 'ECONOMI' in text or 'FINAN' in text: return 'ECONOMIA_FINANZAS'
    if 'DERECHO' in text or 'JURIDIC' in text: return 'DERECHO'
    if 'MEDICINA' in text or 'ENFERMERI' in text or 'SALUD' in text or 'ODONTOLOG' in text: return 'SALUD'
    if 'PSICOLOGI' in text or 'SOCIOLOG' in text: return 'CIENCIAS_SOCIALES'
    if 'DISENO' in text or 'DISEO' in text or 'ARTE' in text or 'MUSICA' in text: return 'ARTE_DISEÑO'
    return 'OTRAS'

def preprocess_data(df, encoders=None, is_training=True, train_columns=None):
    df = df.copy()
    if encoders is None: encoders = {}

    cols_to_drop = ['E_PRIVADO_LIBERTAD', 'F_TIENEINTERNET.1']
    df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

    df['E_PRGM_ACADEMICO'] = (
        df['E_PRGM_ACADEMICO'].astype(str).str.upper()
        .str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
        .str.replace('[^A-Z ]', '', regex=True).str.strip()
    )

    df['CLUSTER_CARRERA'] = df['E_PRGM_ACADEMICO'].apply(get_career_cluster)

    bienes = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'F_TIENECOMPUTADOR']
    for col in bienes:
        if col in df.columns: df[col] = df[col].fillna('No')

    df['F_EDUCACIONPADRE'] = df['F_EDUCACIONPADRE'].fillna('No sabe').replace(['No Aplica'], 'No sabe')
    df['F_EDUCACIONMADRE'] = df['F_EDUCACIONMADRE'].fillna('No sabe').replace(['No Aplica'], 'No sabe')
    df['E_HORASSEMANATRABAJA'] = df['E_HORASSEMANATRABAJA'].fillna('0')
    df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].fillna('Sin Estrato')
    df['E_VALORMATRICULAUNIVERSIDAD'] = df['E_VALORMATRICULAUNIVERSIDAD'].fillna('No pagó matrícula')
    df['E_PAGOMATRICULAPROPIO'] = df['E_PAGOMATRICULAPROPIO'].fillna('No')

    binary_map = {'Si': 1, 'No': 0}
    binary_cols = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'F_TIENECOMPUTADOR', 'E_PAGOMATRICULAPROPIO']

    for col in binary_cols:
        if col in df.columns:
            df[col] = df[col].map(binary_map).astype('uint8')

    valor_matricula_map = {
        'Menos de 500 mil': 0.25,
        'Entre 500 mil y menos de 1 millón': 0.75,
        'Entre 1 millón y menos de 2.5 millones': 1.75,
        'Entre 2.5 millones y menos de 4 millones': 3.25,
        'Entre 4 millones y menos de 5.5 millones': 4.75,
        'Entre 5.5 millones y menos de 7 millones': 6.25,
        'Más de 7 millones': 8.0,
        'No pagó matrícula': 0.0
    }
    df['E_VALORMATRICULAUNIVERSIDAD'] = df['E_VALORMATRICULAUNIVERSIDAD'].map(valor_matricula_map).astype('float32')

    estrato_map = {
        'Sin Estrato': 0,
        'Estrato 1': 1,
        'Estrato 2': 2,
        'Estrato 3': 3,
        'Estrato 4': 4,
        'Estrato 5': 5,
        'Estrato 6': 6
    }
    df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].map(estrato_map).astype('uint8')

    horas_trabajo_map = {
        '0': 0.0,
        'Menos de 10 horas': 5.0,
        'Entre 11 y 20 horas': 15.5,
        'Entre 21 y 30 horas': 25.5,
        'Más de 30 horas': 40.0
    }
    df['E_HORASSEMANATRABAJA'] = df['E_HORASSEMANATRABAJA'].map(horas_trabajo_map).astype('float32')

    educacion_map = {
        'Ninguno': 0,
        'No sabe': 0,
        'Primaria incompleta': 1,
        'Primaria completa': 2,
        'Secundaria (Bachillerato) incompleta': 3,
        'Secundaria (Bachillerato) completa': 4,
        'Técnica o tecnológica incompleta': 5,
        'Técnica o tecnológica completa': 6,
        'Educación profesional incompleta': 7,
        'Educación profesional completa': 8,
        'Postgrado': 9
    }

    df['F_EDUCACIONPADRE'] = df['F_EDUCACIONPADRE'].map(educacion_map).astype('uint8')
    df['F_EDUCACIONMADRE'] = df['F_EDUCACIONMADRE'].map(educacion_map).astype('uint8')

    if 'PERIODO_ACADEMICO' in df.columns:
        df['ANIO'] = (df['PERIODO_ACADEMICO'] // 10).astype('uint16')
        df['SEMESTRE'] = (df['PERIODO_ACADEMICO'] % 10).astype('uint8')
        df = df.drop(columns=['PERIODO_ACADEMICO'])

    target_col_exists = 'RENDIMIENTO_GLOBAL' in df.columns

    if target_col_exists:
        rendimiento_map = {
            'bajo': 0,
            'medio-bajo': 1,
            'medio-alto': 2,
            'alto': 3
        }
        df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(rendimiento_map).astype('uint8')

    if is_training and target_col_exists:
        global_stats = df.groupby('E_PRGM_ACADEMICO')['RENDIMIENTO_GLOBAL'].agg(['mean', 'count'])
        prior = df['RENDIMIENTO_GLOBAL'].mean()
        min_samples = 10
        smoothed = (global_stats['count'] * global_stats['mean'] + min_samples * prior) / (global_stats['count'] + min_samples)
        encoders['E_PRGM_ACADEMICO_TARGET'] = smoothed.to_dict()
        encoders['global_mean'] = prior

        df['E_PRGM_ACADEMICO_ENCODED'] = np.nan
        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        for train_idx, val_idx in kf.split(df):
            X_tr, X_val = df.iloc[train_idx], df.iloc[val_idx]

            fold_stats = X_tr.groupby('E_PRGM_ACADEMICO')['RENDIMIENTO_GLOBAL'].agg(['mean', 'count'])
            fold_prior = X_tr['RENDIMIENTO_GLOBAL'].mean()

            fold_smoothed = (fold_stats['count'] * fold_stats['mean'] + min_samples * fold_prior) / (fold_stats['count'] + min_samples)

            df.loc[val_idx, 'E_PRGM_ACADEMICO_ENCODED'] = X_val['E_PRGM_ACADEMICO'].map(fold_smoothed)

        df['E_PRGM_ACADEMICO_ENCODED'] = df['E_PRGM_ACADEMICO_ENCODED'].fillna(prior)

    else:
        mapping = encoders.get('E_PRGM_ACADEMICO_TARGET', {})
        global_mean = encoders.get('global_mean', 1.5)
        df['E_PRGM_ACADEMICO_ENCODED'] = df['E_PRGM_ACADEMICO'].map(mapping).fillna(global_mean)

    if is_training:
        freq_enc = df['E_PRGM_ACADEMICO'].value_counts(normalize=True).to_dict()
        encoders['E_PRGM_ACADEMICO_FREQ'] = freq_enc
        df['E_PRGM_ACADEMICO_FREQ'] = df['E_PRGM_ACADEMICO'].map(freq_enc)
    else:
        df['E_PRGM_ACADEMICO_FREQ'] = df['E_PRGM_ACADEMICO'].map(encoders.get('E_PRGM_ACADEMICO_FREQ', {})).fillna(0)

    cols_to_dummy = ['E_PRGM_DEPARTAMENTO', 'CLUSTER_CARRERA']
    cols_to_dummy = [c for c in cols_to_dummy if c in df.columns]

    df = pd.get_dummies(df, columns=cols_to_dummy, dtype='uint8')

    if 'E_PRGM_ACADEMICO' in df.columns:
        df = df.drop(columns=['E_PRGM_ACADEMICO'])

    if is_training:
        train_columns = [c for c in df.columns if c != 'RENDIMIENTO_GLOBAL']
        encoders['TRAIN_COLUMNS'] = train_columns
    else:
        train_cols = encoders.get('TRAIN_COLUMNS')
        if train_cols:
            cols_to_keep = df.columns.intersection(train_cols)

            df_aligned = pd.DataFrame(0, index=df.index, columns=train_cols)
            for c in cols_to_keep:
                df_aligned[c] = df[c]

            if target_col_exists:
                df_aligned['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL']

            df = df_aligned

    return df, encoders

### Aplicación del preprocesado a nuestro conjunto de datos

In [ ]:
df, encoders = preprocess_data(df)
df.head()

,ID,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,...,E_PRGM_DEPARTAMENTO_VAUPES,CLUSTER_CARRERA_ADMINISTRACION,CLUSTER_CARRERA_ARTE_DISEÑO,CLUSTER_CARRERA_CIENCIAS_SOCIALES,CLUSTER_CARRERA_DERECHO,CLUSTER_CARRERA_ECONOMIA_FINANZAS,CLUSTER_CARRERA_EDUCACION,CLUSTER_CARRERA_INGENIERIA,CLUSTER_CARRERA_OTRAS,CLUSTER_CARRERA_SALUD
0,904256,6.25,5.0,3,1,5,1,1,0,1,...,0,0,0,0,0,0,0,0,0,1
1,645256,3.25,0.0,3,0,6,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,308367,3.25,40.0,3,1,4,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,470353,4.75,0.0,4,1,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0
4,989032,3.25,25.5,3,1,2,1,1,0,1,...,0,0,0,1,0,0,0,0,0,0


## Split de los datos para entrenamiento y test

In [ ]:
y_col = 'RENDIMIENTO_GLOBAL'

X = df.drop(columns=[y_col, 'ID'])
y = df[y_col]
X.shape, y.shape

((692500, 58), (692500,))

In [ ]:
Xtr, Xts, ytr, yts = train_test_split(X,y, train_size=0.8, random_state=42, stratify=y)
Xtr.shape, Xts.shape, ytr.shape, yts.shape

((554000, 58), (138500, 58), (554000,), (138500,))

## Entrenamiento y prueba del Modelo CatBoost Classifier

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

cat_model = CatBoostClassifier(
    loss_function='MultiClass',
    eval_metric='Accuracy',
    iterations=3000,
    learning_rate=0.015,
    depth=9,
    l2_leaf_reg=5,
    border_count=128,
    bagging_temperature=1,
    early_stopping_rounds=200,
    random_seed=42,
    verbose=200
)


cat_model.fit(
    Xtr, y=ytr,
    eval_set=(Xts, yts),
    use_best_model=True,
    plot=False
)

0:	learn: 0.3968357	test: 0.3949097	best: 0.3949097 (0)	total: 1.45s	remaining: 1h 12m 23s
200:	learn: 0.4289170	test: 0.4248881	best: 0.4248881 (200)	total: 3m 42s	remaining: 51m 32s
400:	learn: 0.4366697	test: 0.4309531	best: 0.4309531 (400)	total: 7m 9s	remaining: 46m 23s
600:	learn: 0.4426462	test: 0.4339928	best: 0.4341083 (598)	total: 10m 28s	remaining: 41m 48s
800:	learn: 0.4476245	test: 0.4357401	best: 0.4357401 (800)	total: 13m 50s	remaining: 37m 59s
1000:	learn: 0.4522978	test: 0.4372708	best: 0.4372924 (998)	total: 17m 9s	remaining: 34m 15s
1200:	learn: 0.4562256	test: 0.4383682	best: 0.4384549 (1195)	total: 20m 25s	remaining: 30m 36s
1400:	learn: 0.4595361	test: 0.4386065	best: 0.4387942 (1315)	total: 23m 48s	remaining: 27m 10s
1600:	learn: 0.4629242	test: 0.4390181	best: 0.4392924 (1568)	total: 27m 12s	remaining: 23m 46s
1800:	learn: 0.4664097	test: 0.4392563	best: 0.4395307 (1744)	total: 30m 29s	remaining: 20m 18s
2000:	learn: 0.4697635	test: 0.4395957	best: 0.4398267 (19

### Test del modelo entrenado

In [ ]:
y_pred = cat_model.predict(Xts)
accuracy_score(yts, y_pred)

0.44010108303249096

## Cargar dataset de prueba para Kaggle y preprocesarlo

In [ ]:
test_df = pd.read_csv("test.csv")

## Preprocesamos el dataset con los encoders
test_df, _ = preprocess_data(test_df, encoders=encoders, is_training=False)

ids = test_df['ID']
test_df = test_df.drop(columns=['ID'])

### Hacer las predicciones sobre `test.csv`

In [ ]:
predictions = cat_model.predict(test_df.values)
predictions = predictions.ravel()

cmap = {
    0: 'bajo',
    1: 'medio-bajo',
    2: 'medio-alto',
    3: 'alto'
}

predictions = pd.Series(predictions).map(cmap)
result = pd.DataFrame({
    'ID': ids,
    'RENDIMIENTO_GLOBAL': predictions
})
result.to_csv('predictions.csv', index=False)